#SMT

In [1]:
!git clone https://github.com/FrancescoFarinola/VLSI_CDMO
!pip install z3-solver
from z3 import *

fatal: destination path 'VLSI_CDMO' already exists and is not an empty directory.


You should consider upgrading via the 'c:\users\francesco.farinola\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
from VLSI_CDMO.utils import input, output
import os
import numpy as np

#Model

In [3]:
def bound_coordinates(coordinate, size, max):
    '''
    Generic decomposition of the implied constraints to define the lower and upper bounds of the output coordinates 
    Calls to this function have to be made with:
        - x, width and plate_w
        - y, height and plate_h
    @Args:
    - coordinate : indicates the coordinate on the axis (either x or y) (IntVector)
    - size : indicates the size of the axis (either width or height) (array of int)
    - max : indicates the upper bound for values taken by coordinate (either plate_w or plate_h) (int)
    '''
    decomposition = []
    for i in range(len(coordinate)):
        decomposition.append(And(coordinate[i] >= 0,              # Lower bound
                                 coordinate[i] + size[i] <= max)) # Upper bound
    return decomposition

def diffn(x, y, width, height):
    '''
    This function defines the decomposition of the diffn global constraint in order to assure
    no overlap between circuits. Simply sum between the coordinate x and width x of a circuit 
    has to be lesser or equal than the coordinate x of another circuit.
    @Args:
    - x : array of x coordinates (IntVector)
    - y : array of y coordinates (IntVector)
    - width : array of widths of the circuits (array of int)
    - height : array of heights of the circuits (array of int)
    '''
    decomposition = []
    n = len(x)
    for i in range(n):
        for j in range(i+1,n):
            decomposition.append(Or([x[i] + width[i]  <= x[j],
                                     y[i] + height[i] <= y[j],
                                     x[j] + width[j]  <= x[i],
                                     y[j] + height[j] <= y[i]]))
    return decomposition

'''
def cumulative(start, duration, resources, total):
    Requires that a set of tasks given by start times, durations, and resource requirements, 
    never require more than a global resource bound 'total' at any one time.
    This can be adapted to our case in which start times can be seen as coordinates x (respectively y),
    duration can be seen as the width of rectangles (respectively height), resources as the height
    (respectively width) and the total amount of resources as the maximum height (respectively
    the maximum width) of the plate
    @Args:
    - start : array of coordinates (IntVector)
    - duration : size of circuits (array of int)
    - resources : size of circuits (array of int)
    - total : maximum size of the plate (int)
    
    decomposition = []
    tasks = [i for i in range(len(start)) if resources[i] > 0 and duration[i] > 0] #Assumptions of cumulative constrain r[i] and d[i]>0
    for i in tasks:
        decomposition.append(
            Sum([If(And(start[i] <= resources[i], resources[i] < start[j] + duration[j]), resources[j], 0)
                 for j in range(len(start))]) <= total
        )
    return decomposition
'''





def force_adjacency(coordinate, size):
    decomposition = []
    for i in range(len(coordinate)):
        lst = [0]
        for j in range(len(coordinate)):
            if i != j:
                lst.append(coordinate[j]+size[j])                   # Creates list of possible adjacency values 
        decomposition.append(Or([coordinate[i] == j for j in lst])) # Encoding of Member constraint
    return decomposition


def same_dimensions(x, y, width, height):
    decomposition = []
    for i in range(len(width)):
        for j in range(i+1, len(width)):
            decomposition.append(Implies(And(width[i] == width[j], height[i] == height[j]),
                                         Or(And(x[i] <  x[j], y[i] <= y[j]),
                                            And(x[i] <= x[j], y[i] <  y[j]))))
    return decomposition


def place_biggest_circuit(x, y, area):
    return [Or(And(x[area[0]] <  x[area[1]], y[area[0]] <= y[area[1]]), And(x[area[0]] <= x[area[1]], y[area[0]] <  y[area[0]]))]


In [ ]:
def write_output(filename, plate_w):
    out_file = "out/" + os.path.basename(filename).replace('ins', 'out')
    with open(out_file, 'w') as f:
        f.write("{0} {1}\n".format(instance["plate_w"], result["plate_h"]))
        f.write("{0}\n".format(instance["n"]))
        for i in range(0,instance["n"]):
            if rotation:
                if result["rotation"][i]:
                    f.write("{0} {1} {2} {3}\n".format(instance["height"][i], instance["width"][i], result["c_x"][i], result["c_y"][i]))
                else:
                    f.write("{0} {1} {2} {3}\n".format(instance["width"][i], instance["height"][i], result["c_x"][i], result["c_y"][i]))
            else:
                f.write("{0} {1} {2} {3}\n".format(instance["width"][i], instance["height"][i], result["c_x"][i], result["c_y"][i]))
    return out_file

In [ ]:
import time
path = "VLSI_CDMO/instances/"
input_files = sorted(sorted([path + i for i in os.listdir(path)], key=len)[:9]) + sorted(sorted([path + i for i in os.listdir(path)], key=len)[9:])
for i in input_files:
  print("Solving instance {0}".format(i))
  plate_w, n, width, height = input.read_instance(i)

  s = Optimize()

  plate_h = Int('plate_h')
  x = IntVector('x', n)
  y = IntVector('y', n)

  areas = sorted(np.argsort([height[i] * width[i] for i in range(n)]), reverse=True) #indexes of circuits sorted by decreasing area


  s.add(bound_coordinates(x, width, plate_w) +
        bound_coordinates(y, height, plate_h) +
        diffn(x, y, width, height) +
        #cumulative1(x, width, height, plate_h) + 
        #cumulative1(y, height, width, plate_w) + 
        force_adjacency(x, width) +
        force_adjacency(y, height) +
        same_dimensions(x, y, width, height) +
        place_biggest_circuit(x, y, areas)
  )
  s.set(timeout=300*1000)
  s.assertions()
  s.minimize(plate_h)
  start_time = time.time()
  if s.check() == sat:
      m = s.model()
      circuits_pos = [(wi, hi, m.eval(xi).as_long(), m.eval(yi).as_long()) for wi, hi, xi, yi in zip(width, height, x, y)]
      h = m.eval(plate_h).as_long()

      print(plate_w, h, circuits_pos, time.time() - start_time)
  else:
      print("No solution found - UNSAT")

Solving instance VLSI_CDMO/instances/ins-1.txt
8 8 [(3, 3, 0, 5), (3, 5, 5, 0), (5, 3, 3, 5), (5, 5, 0, 0)] 0.007978677749633789
Solving instance VLSI_CDMO/instances/ins-2.txt
9 9 [(3, 3, 3, 0), (3, 4, 6, 0), (3, 5, 6, 4), (3, 6, 3, 3), (3, 9, 0, 0)] 0.018949031829833984
Solving instance VLSI_CDMO/instances/ins-3.txt
10 10 [(3, 3, 0, 0), (3, 4, 3, 6), (3, 6, 7, 0), (3, 7, 0, 3), (4, 4, 6, 6), (4, 6, 3, 0)] 0.03989410400390625
Solving instance VLSI_CDMO/instances/ins-4.txt
11 11 [(3, 3, 3, 8), (3, 4, 3, 4), (3, 5, 0, 0), (3, 6, 0, 5), (5, 3, 6, 4), (5, 4, 6, 7), (8, 4, 3, 0)] 0.04787254333496094
Solving instance VLSI_CDMO/instances/ins-5.txt
12 12 [(3, 3, 3, 3), (3, 4, 9, 8), (3, 5, 0, 0), (3, 6, 3, 6), (3, 7, 0, 5), (3, 8, 9, 0), (3, 9, 6, 3), (6, 3, 3, 0)] 0.13867402076721191
Solving instance VLSI_CDMO/instances/ins-6.txt
13 13 [(3, 3, 0, 6), (3, 4, 0, 9), (3, 5, 10, 8), (3, 6, 7, 7), (3, 7, 7, 0), (3, 8, 10, 0), (4, 3, 3, 10), (4, 4, 3, 6), (7, 6, 0, 0)] 0.23035144805908203
Solving i